In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('/content/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# checking impact of columns on outcome
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
# Extracting X and y colums from data
X = df.iloc[:, 0:-1].values
y = df.iloc[:, -1].values

In [7]:
# Scaling our data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [8]:
print(X)
print(X.shape)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
(768, 8)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [63]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [11]:
# Creating model architecture without using keras tuner
# so we have to create manually
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.6577 - accuracy: 0.6547 - val_loss: 0.6077 - val_accuracy: 0.6818
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6216 - accuracy: 0.6954 - val_loss: 0.5755 - val_accuracy: 0.7727
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5938 - accuracy: 0.7215 - val_loss: 0.5527 - val_accuracy: 0.7987
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5735 - accuracy: 0.7329 - val_loss: 0.5322 - val_accuracy: 0.7987
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5538 - accuracy: 0.7394 - val_loss: 0.5167 - val_accuracy: 0.7922
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5391 - accuracy: 0.7443 - val_loss: 0.5026 - val_accuracy: 0.7922
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5259 - accuracy: 0.7541 - val_loss: 0.4924 - val_accuracy: 0.7857
Epoch 8/100


In [13]:
# 1. How to select appropriate optimizer
# 2. Select No. of nodes in layer
# 3. How to select No. of Hidden layers
# 4. All in one model creating

In [15]:
# importing keras tuner
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [20]:
import kerastuner as kt

In [21]:
# 1. How to select appropriate optimizer
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [22]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [23]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4224 - accuracy: 0.8013 - val_loss: 0.4481 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4221 - accuracy: 0.7997 - val_loss: 0.4482 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4225 - accuracy: 0.8029 - val_loss: 0.4477 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4214 - accuracy: 0.8013 - val_loss: 0.4474 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4219 - accuracy: 0.7997 - val_loss: 0.4475 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4207 - accuracy: 0.8013 - val_loss: 0.4476 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4209 - accuracy: 0.8046 - val_loss: 0.4490 - val_accuracy: 0.8117
Epoch 14/

In [31]:
# 2. Select No. of nodes in layer
def build_model(hp):
  model = Sequential()

  units = hp.Int('units', min_value=8,max_value=128)
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model


In [32]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='my_dir', project_name='ParameterTune')

In [33]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 09s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'units': 55}

In [35]:
model = tuner.get_best_models(num_models=1)[0]

In [36]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=5)

Epoch 6/100
20/20 [==============================] - 1s 4ms/step - loss: 0.4882 - accuracy: 0.7785
Epoch 7/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4791 - accuracy: 0.7850
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4715 - accuracy: 0.7818
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4660 - accuracy: 0.7899
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4618 - accuracy: 0.7915
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4581 - accuracy: 0.7883
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4561 - accuracy: 0.7932
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4537 - accuracy: 0.7866
Epoch 14/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4523 - accuracy: 0.7850
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4507 - accuracy: 0.7850
Epoc

In [40]:
# 3. How to select No. of Hidden layers
def build_model(hp):
  model = Sequential()

  model.add(Dense(72, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [41]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='my_dir', project_name='Num_layers')

In [42]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 14s


In [43]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [46]:
model = tuner.get_best_models(num_models=1)[0]

In [49]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.4479 - accuracy: 0.7818 - val_loss: 0.4735 - val_accuracy: 0.8182
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4372 - accuracy: 0.7932 - val_loss: 0.4818 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4298 - accuracy: 0.8013 - val_loss: 0.4771 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4228 - accuracy: 0.7980 - val_loss: 0.4848 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4098 - accuracy: 0.8160 - val_loss: 0.4886 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4094 - accuracy: 0.8111 - val_loss: 0.4981 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4003 - accuracy: 0.8013 - val_loss: 0.5016 - val_accuracy: 0.7922
Epoch 14

In [72]:
# 4. All in one model creating
def build_model(hp):
  model = Sequential()
  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=1, max_value=120, step=8),
              activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.9])))
    else:
      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=1, max_value=120, step=8),
              activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.9])))

      counter+=1

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=hp.Choice('optimizer', values=['rmsprop','adam','sgd','nadam','adadelta']),
        loss='binary_crossentropy', metrics=['accuracy']
        )

  return model

In [73]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='my_dir', project_name='final1')

In [74]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.701298713684082
Total elapsed time: 00h 01m 14s


In [75]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 17,
 'activation0': 'relu',
 'dropout0': 0.7,
 'optimizer': 'nadam',
 'units1': 9,
 'activation1': 'relu',
 'dropout1': 0.6,
 'units2': 9,
 'activation2': 'sigmoid',
 'dropout2': 0.5,
 'units3': 97,
 'activation3': 'sigmoid',
 'dropout3': 0.7,
 'units4': 65,
 'activation4': 'tanh',
 'dropout4': 0.6,
 'units5': 1,
 'activation5': 'relu',
 'dropout5': 0.3,
 'units6': 33,
 'activation6': 'relu',
 'dropout6': 0.6,
 'units7': 97,
 'activation7': 'tanh',
 'dropout7': 0.1,
 'units8': 9,
 'activation8': 'relu',
 'dropout8': 0.1}

In [76]:
model = tuner.get_best_models(num_models=1)[0]

In [77]:
model.fit(X_train, y_train, epochs=200, initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/200
20/20 [==============================] - 1s 12ms/step - loss: 0.6792 - accuracy: 0.6238 - val_loss: 0.6210 - val_accuracy: 0.7078
Epoch 7/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6729 - accuracy: 0.6189 - val_loss: 0.6132 - val_accuracy: 0.7078
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6637 - accuracy: 0.6466 - val_loss: 0.6058 - val_accuracy: 0.7273
Epoch 9/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6764 - accuracy: 0.6336 - val_loss: 0.5980 - val_accuracy: 0.7208
Epoch 10/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6222 - accuracy: 0.6743 - val_loss: 0.5919 - val_accuracy: 0.7273
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6333 - accuracy: 0.6792 - val_loss: 0.5853 - val_accuracy: 0.7338
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6327 - accuracy: 0.6694 - val_loss: 0.5788 - val_accuracy: 0.7338
Epoch 13/

In [80]:
! rm -r /content/my_dir/final